# 0. Load packages

In [197]:
import os,sys,json,time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp
import plotly
import plotly.express as px 
from collections import defaultdict
import calendar



# 1. Load Data [PYSNOOPER, Polaris, Categories]

In [198]:
polaris_data = pd.read_csv('/Users/tpapka/Summer2025/PyModuleSnooper/polaris_job_location_20250609.csv')

pysnooper = pd.read_csv('/Users/tpapka/Summer2025/module_data/pythonlogging/module_usage/output.csv.gz',compression='gzip')








with open('/Users/tpapka/Summer2025/PYMODULESNOOPER/newCatergories.json') as file:
   categories =json.load(file)

#If additons are made to categories run this to check for duplicates

# library_to_categories = defaultdict(list)
# for category, libs in categories.items():
#     for lib in libs:
#         library_to_categories[lib].append(category)

# duplicates = {lib: cats for lib, cats in library_to_categories.items() if len(cats) > 1}

# if duplicates:
#     print("Duplicates found:")
#     for lib, cats in sorted(duplicates.items()):
#         print(f"  '{lib}' appears in categories: {', '.join(cats)}")
# else:
#     print("No duplicates found.")

/var/folders/33/hsn1szm57t10kpxgfznwvncw0000gn/T/ipykernel_27555/352124516.py:1: DtypeWarning:

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.



In [199]:
polaris_data['START_TIMESTAMP'].sort_values()

671373    1970-01-01 00:00:00
671374    1970-01-01 00:00:00
0         2021-11-16 19:42:44
363       2021-11-16 19:42:44
364       2021-11-16 19:42:44
                 ...         
934893    2025-06-09 19:06:06
934896    2025-06-09 19:07:05
934897    2025-06-09 19:09:07
934898    2025-06-09 19:10:07
934900    2025-06-09 19:11:08
Name: START_TIMESTAMP, Length: 934901, dtype: object

In [200]:
# polaris_data['START_TIMESTAMP'] = pd.to_datetime(polaris_data['START_TIMESTAMP'], errors='coerce')

# condition = (
#     (polaris_data['START_TIMESTAMP'] == pd.Timestamp('1970-01-01')) &
#     (polaris_data['JOB_NAME'] == '1983946.polaris')
# )

# polaris_data.loc[condition, 'START_TIMESTAMP'] = pd.Timestamp('2024-06-14 21:17:41.0')

# condition = (
#     (polaris_data['START_TIMESTAMP'] == pd.Timestamp('1970-01-01')) &
#     (polaris_data['JOB_NAME'] == '1983947.polaris')
# )
# polaris_data.loc[condition, 'START_TIMESTAMP'] = pd.Timestamp('2024-06-14 21:18:30.0')





polaris_data = polaris_data[polaris_data['START_TIMESTAMP'] >= '2023-12-25']


In [201]:
polaris_data[polaris_data['START_TIMESTAMP'] == '1970-01-01 00:00:00']

,END_DATE_ID,JOB_NAME,PROJECT_NAME,USERNAME,SCIENCE_FIELD_SHORT,QUEUED_TIMESTAMP,QUEUED_DATE_ID,START_TIMESTAMP,START_DATE_ID,END_TIMESTAMP,END_DATE_ID.1,NODES_USED,WALLTIME_SECONDS,RUNTIME_SECONDS,LOCATION


# 2. Seperate login and compute nodes from PYSNOOPER & Add Conda Column

In [202]:
login_node = 'polaris-login'

pysnooper['node'] = pysnooper['hostname'].apply(lambda x: 'login_node' if login_node in x else 'compute_node')

pysnooper['modules'] = pysnooper['modules'].apply(eval)


pysnooper_compute = pysnooper[pysnooper['node'] == 'compute_node']

pysnooper_login = pysnooper[pysnooper['node'] == 'login_node']

conda_env = '/lus/theta-fs0/software/applications/conda'
pysnooper_compute['conda'] = pysnooper_compute['source'].apply(lambda x: 'Yes' if conda_env in x else 'No')




/var/folders/33/hsn1szm57t10kpxgfznwvncw0000gn/T/ipykernel_27555/534047387.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# 3. Merge Pysnooper compute node data with all of Polaris data

In [203]:
# Prepare all_jobs
polaris_data['START_TIMESTAMP'] = pd.to_datetime(polaris_data['START_TIMESTAMP'])
polaris_data['END_TIMESTAMP'] = pd.to_datetime(polaris_data['END_TIMESTAMP'])
pysnooper_compute['timestamp'] = pd.to_datetime(pysnooper_compute['timestamp'])


polaris_data['LOCATION'] = polaris_data['LOCATION'].str.split(',')
polaris_data = polaris_data.explode('LOCATION')
polaris_data['LOCATION'] = polaris_data['LOCATION'].astype(str)

# all_jobs = all_jobs.sort_values(by = 'START_TIMESTAMP', ascending=True)
pysnooper_compute = pysnooper_compute.sort_values('timestamp', ascending=True)

# Setup batching
chunk_size = 100_000
results = []

for i in range(0, len(pysnooper_compute), chunk_size):
    print(f"Processing rows {i} to {i + chunk_size}...")

    chunk = pysnooper_compute.iloc[i:i+chunk_size].copy()
    chunk['timestamp'] = pd.to_datetime(chunk['timestamp'])
    chunk['hostname'] = chunk['hostname'].astype(str)

    # Filter jobs to relevant time range for the chunk
    min_time = chunk['timestamp'].min()
    max_time = chunk['timestamp'].max()
    jobs_subset = polaris_data[
        (polaris_data['END_TIMESTAMP'] >= min_time) &
        (polaris_data['START_TIMESTAMP'] <= max_time)
    ]

    # Merge and filter
    merged = chunk.merge(
        jobs_subset,
        left_on='hostname',
        right_on='LOCATION',
        how='left'
    )

    matched = merged[
        (merged['timestamp'] >= merged['START_TIMESTAMP']) &
        (merged['timestamp'] <= merged['END_TIMESTAMP'])
    ]
    matched = (
    matched.sort_values('START_TIMESTAMP')
    .drop_duplicates(subset=['timestamp', 'hostname'], keep='first')
    )
    print("Merged shape:", merged.shape)
    print("Matching timestamps:", ((merged['timestamp'] >= merged['START_TIMESTAMP']) & (merged['timestamp'] <= merged['END_TIMESTAMP'])).sum())
    


    results.append(
        matched[[
            'timestamp', 'hostname', 'modules', 'NODES_USED', 'END_DATE_ID', 'JOB_NAME', 'PROJECT_NAME', 'USERNAME', 'SCIENCE_FIELD_SHORT', 'QUEUED_TIMESTAMP', 'QUEUED_DATE_ID','START_TIMESTAMP', 'START_DATE_ID', 'END_TIMESTAMP', 'WALLTIME_SECONDS', 'RUNTIME_SECONDS', 'LOCATION','conda'
        ]]
    )

# Combine all chunk results
final_df = pd.concat(results, ignore_index=True)
print("Final shape:", final_df.shape)
print(final_df.head())


/var/folders/33/hsn1szm57t10kpxgfznwvncw0000gn/T/ipykernel_27555/3624430337.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Processing rows 0 to 100000...
Merged shape: (42477041, 27)
Matching timestamps: 86613
Processing rows 100000 to 200000...
Merged shape: (22849272, 27)
Matching timestamps: 84274
Processing rows 200000 to 300000...
Merged shape: (1841552, 27)
Matching timestamps: 96306
Processing rows 300000 to 400000...
Merged shape: (1880243, 27)
Matching timestamps: 94408
Processing rows 400000 to 500000...
Merged shape: (7608689, 27)
Matching timestamps: 86571
Processing rows 500000 to 600000...
Merged shape: (1752559, 27)
Matching timestamps: 76262
Processing rows 600000 to 700000...
Merged shape: (5733752, 27)
Matching timestamps: 84914
Processing rows 700000 to 800000...
Merged shape: (56154268, 27)
Matching timestamps: 94875
Processing rows 800000 to 900000...
Merged shape: (29346956, 27)
Matching timestamps: 98097
Processing rows 900000 to 1000000...
Merged shape: (46679436, 27)
Matching timestamps: 99507
Processing rows 1000000 to 1100000...
Merged shape: (136793567, 27)
Matching timestamps: 

### 3.5 Post processing after merge (Adding: Date, node_size)

In [204]:
merged_df = final_df
merged_df['date'] = pd.to_datetime(merged_df['timestamp']).dt.date
def node_level(x):
   if 10 >= x >= 1:
      return 'Small'
   elif 25 >= x >= 11:
      return 'Medium'
   else:
      return 'Large'
merged_df['node_size'] = merged_df['NODES_USED'].apply(node_level)


In [205]:
merged_df

,timestamp,hostname,modules,NODES_USED,END_DATE_ID,JOB_NAME,PROJECT_NAME,USERNAME,SCIENCE_FIELD_SHORT,QUEUED_TIMESTAMP,QUEUED_DATE_ID,START_TIMESTAMP,START_DATE_ID,END_TIMESTAMP,WALLTIME_SECONDS,RUNTIME_SECONDS,LOCATION,conda,date,node_size
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,"[ipaddress, boltons, _distutils_hack, _typing,...",1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small
1,2024-04-30 09:56:17.296203,x3210c0s25b0n0,"[Cython, accelerate, ipaddress, pydot, _cspars...",1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,No,2024-04-30,Small
2,2024-04-30 09:56:17.273981,x3210c0s25b0n0,"[Cython, accelerate, ipaddress, pydot, _cspars...",1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,No,2024-04-30,Small
3,2024-04-30 09:56:17.251766,x3210c0s25b0n0,"[Cython, accelerate, ipaddress, pydot, _cspars...",1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,No,2024-04-30,Small
4,2024-04-30 09:56:14.396731,x3210c0s1b0n0,"[Cython, accelerate, ipaddress, pydot, _cspars...",1.0,20240430.0,1878285.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:39,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s1b0n0,No,2024-04-30,Small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965172,2024-12-29 14:50:15.321005,x3204c0s1b1n0,"[Cython, ipaddress, pydot, asyncio, onnx, ligh...",32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,20241221.0,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large
965173,2024-12-29 14:50:15.266531,x3204c0s1b1n0,"[Cython, ipaddress, pydot, asyncio, onnx, ligh...",32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,20241221.0,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large
965174,2024-12-29 14:50:15.236127,x3204c0s1b1n0,"[Cython, ipaddress, pydot, asyncio, onnx, ligh...",32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,20241221.0,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large
965175,2024-12-29 14:50:08.081899,x3203c0s1b0n0,"[ipaddress, boltons, _distutils_hack, _typing,...",32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,20241221.0,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3203c0s1b0n0,Yes,2024-12-29,Large


# 4. Create seperate dataset with modules exploded 

In [206]:
merged_df_exploded = merged_df.explode('modules')
merged_df_exploded

,timestamp,hostname,modules,NODES_USED,END_DATE_ID,JOB_NAME,PROJECT_NAME,USERNAME,SCIENCE_FIELD_SHORT,QUEUED_TIMESTAMP,QUEUED_DATE_ID,START_TIMESTAMP,START_DATE_ID,END_TIMESTAMP,WALLTIME_SECONDS,RUNTIME_SECONDS,LOCATION,conda,date,node_size
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,ipaddress,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,boltons,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,_distutils_hack,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,_typing,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,ruamel,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,20240427.0,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,sympy,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,20241221.0,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,ruamel,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,20241221.0,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,zipimport,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,20241221.0,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,jsonargparse,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,20241221.0,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large


# 5. Connect modules with categories on exploded df

In [207]:
module_to_category = {}
for category, modules in categories.items():
    for module in modules:
        module_to_category[module] = category

merged_df_exploded['category'] = merged_df_exploded['modules'].map(module_to_category).fillna('unknown')

merged_df_exploded

,timestamp,hostname,modules,NODES_USED,END_DATE_ID,JOB_NAME,PROJECT_NAME,USERNAME,SCIENCE_FIELD_SHORT,QUEUED_TIMESTAMP,...,START_TIMESTAMP,START_DATE_ID,END_TIMESTAMP,WALLTIME_SECONDS,RUNTIME_SECONDS,LOCATION,conda,date,node_size,category
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,ipaddress,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,stdlibs
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,boltons,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,utilities
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,_distutils_hack,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,stdlibs
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,_typing,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,stdlibs
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,ruamel,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,parsing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,sympy,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,...,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large,ds
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,ruamel,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,...,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large,parsing
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,zipimport,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,...,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large,stdlibs
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,jsonargparse,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,...,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large,config


In [208]:
mistakes = polaris_data[polaris_data['START_TIMESTAMP'] == '1970-01-01']
mistakes['PROJECT_NAME'].unique()

array([], dtype=object)

# 6. Analysis

# 6.0 General Viz

### a). Jobs by Day Bar chart

In [209]:
date_count = merged_df.groupby('date')['date'].count().rename('Count').reset_index()
fig = px.bar(date_count, x = 'date', y = 'Count', title = 'Pysnooper Jobs Ran Each Day 2024', labels = {
   'date':'Date'
})
fig.show()

### b). PySnooper Jobs vs All Polaris Jobs 

In [210]:
polaris_data['START_TIMESTAMP'] = pd.to_datetime(polaris_data['START_TIMESTAMP'])

polaris_data_2024 = polaris_data[polaris_data['START_TIMESTAMP'] >= '2024-01-01'].copy()
polaris_data_2024['month'] = polaris_data_2024['START_TIMESTAMP'].dt.month
unique_jobs_by_month_polaris = (
    polaris_data_2024
    .groupby('month')['JOB_NAME']
    .nunique()
    .reset_index()
    .rename(columns={'JOB_NAME': 'unique_job_count'})
)
pysnooper_monthly = merged_df
pysnooper_monthly['month'] = merged_df['timestamp'].dt.month
unique_jobs_by_months_pysnooper = (pysnooper_monthly.groupby('month')['JOB_NAME'].nunique().reset_index().rename(columns={'JOB_NAME': 'unique_job_count'}))

df1 = unique_jobs_by_month_polaris.rename(columns={'unique_job_count': 'Polaris'})
df2 = unique_jobs_by_months_pysnooper.rename(columns={'unique_job_count': 'PySnooper'})

merged = df1.merge(df2, on='month')

df_long = merged.melt(id_vars='month', value_vars=['Polaris', 'PySnooper'],
                      var_name='Source', value_name='UniqueJobCount')

df_long['month'] = df_long['month'].apply(lambda x: calendar.month_abbr[x])

fig = px.bar(df_long, x='month', y='UniqueJobCount', color='Source',
             barmode='group', title='Unique Jobs by Month: Polaris vs PySnooper',
             labels={'month': 'Month', 'UniqueJobCount': 'Unique Job Count'})

fig.show()
df_long

merged['percent_diff'] = (merged['PySnooper']) / merged['Polaris'] * 100
merged['month'] = merged['month'].apply(lambda x: calendar.month_abbr[x])

percent_diff_df = merged[['month', 'percent_diff']]
print(percent_diff_df)



  month  percent_diff
0   Apr      0.380522
1   May     20.230069
2   Jun     29.904236
3   Jul     40.567859
4   Aug     40.493379
5   Sep     21.963691
6   Oct     18.787833
7   Nov     11.501065
8   Dec      5.692125


# 6.1 Conda Viz

### a). Conda Usage Pie Chart

In [211]:
conda_count = merged_df.groupby('conda')['conda'].count().rename('count').reset_index()

fig = px.pie(conda_count, names = 'conda', values = 'count', color_discrete_map = {
   'yes':'blue',
   'no':'red'
   },
   title = 'Conda Usage 2024', labels = {
      'conda':'Using Conda',
      'count':'Count',
      'yes':'Yes',
      'no':'No'
   }
   )
fig.update_layout(legend_title_text = 'Conda Usage', legend_traceorder = 'reversed')
fig.show()

### b). Conda Usage Over Time Bar Chart

In [212]:
conda_count_dates = merged_df.groupby(['conda','date'])['conda'].count().rename('count').reset_index()

fig = px.bar(conda_count_dates, x = 'date', y = 'count', color = 'conda', color_discrete_map = {
   'Yes':'blue',
   'No':'red'
}, title = 'Conda Usage Over 2024', labels = {
   'date':'Date',
   'count':'Count',
   'conda':'Conda Usage'
   
})
fig.update_layout(legend_traceorder = 'reversed')
fig.show()

# 6.2 Node Size Viz

### a). Node Size Pie Chart 

In [213]:
node_size_count = merged_df.groupby('node_size')['node_size'].count().rename('count').reset_index()

fig = px.pie(node_size_count, names = 'node_size', values = 'count', category_orders = {
   'node_size': ['Small', 'Medium', 'Large']
}, title = 'Node Usage Sizes 2024', labels = {
   'node_size':'Node Size',
   'count':'Count'
})
fig.show()

### b). Node Size Over Time 


In [214]:
node_size_count_date = merged_df.groupby(['node_size', 'date'])['node_size'].count().rename('Count').reset_index()
fig = px.bar(node_size_count_date, x = 'date', y = 'Count', color = 'node_size', title = 'Node Usage Over Time 2024', labels = {
   'date':'Date',
   'node_size':'Node Size'
})
fig.show()

### c). Conda Usage to Node Size Sunburst

In [215]:
conda_node_size_count = merged_df.groupby(['conda','node_size'])['node_size'].count().rename('Count').reset_index()

fig = px.sunburst(conda_node_size_count, path = ['conda', 'node_size'], values = 'Count', title = 'Conda Usage to Node Size 2024', labels = {
   'conda':'Conda',
   'node_size':'Node Size',
})
fig.show()

# 6.3 Category Viz

### a). Category Pie Chart

In [216]:
category_count = merged_df_exploded.groupby('category')['category'].count().rename('Count').reset_index()

fig = px.pie(category_count, names = 'category', values = 'Count', title = 'Categories Used in 2024', labels = {
   'category':'Category',
} )
fig.show()

### b). Categories Over Time

In [217]:
category_count_date = merged_df_exploded.groupby(['category','date'])['category'].count().rename('Count').reset_index()

fig = px.bar(category_count_date, x = 'date', y = 'Count', color = 'category', title = 'Categories Over Time 2024', labels = {
   'category':'Categories',
   'date':'Date'
})
fig.show()

### c). Conda to Node Size to Category Suburst

In [218]:
category_sunburst = merged_df_exploded.groupby(['conda','node_size','category'])['category'].count().rename('Count').reset_index()

fig = px.sunburst(category_sunburst, path = ['conda','node_size','category'], values = 'Count', title = 'Categories by Node Size Conda Usage 2024')
fig.show()

# 6.4 Module Viz

### Often most useful to only viz Top 5 or 10 

In [219]:
merged_df_exploded

,timestamp,hostname,modules,NODES_USED,END_DATE_ID,JOB_NAME,PROJECT_NAME,USERNAME,SCIENCE_FIELD_SHORT,QUEUED_TIMESTAMP,...,START_TIMESTAMP,START_DATE_ID,END_TIMESTAMP,WALLTIME_SECONDS,RUNTIME_SECONDS,LOCATION,conda,date,node_size,category
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,ipaddress,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,stdlibs
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,boltons,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,utilities
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,_distutils_hack,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,stdlibs
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,_typing,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,stdlibs
0,2024-04-30 09:34:32.389648,x3210c0s25b0n0,ruamel,1.0,20240430.0,1878288.polaris,sbi-fair,rongit,Computer Science,2024-04-27 01:27:48,...,2024-04-30 01:36:49,20240430.0,2024-04-30 13:36:59,43200.0,43201.0,x3210c0s25b0n0,Yes,2024-04-30,Small,parsing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,sympy,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,...,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large,ds
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,ruamel,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,...,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large,parsing
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,zipimport,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,...,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large,stdlibs
965176,2024-12-29 14:50:15.560505,x3204c0s1b1n0,jsonargparse,32.0,20241229.0,3118878.polaris,FoundEnergy,abhutani,Energy Technologies,2024-12-21 20:25:00,...,2024-12-29 14:50:01,20241229.0,2024-12-29 14:50:26,21600.0,10.0,x3204c0s1b1n0,No,2024-12-29,Large,config


### a). Total Module Usage by Day Bar Chart

In [220]:
module_dates = merged_df_exploded.groupby('date')['modules'].count().rename('Count').reset_index()
fig = px.bar(module_dates, x = 'date', y = 'Count', title = 'Module Usage 2024', labels = {
   'date':'Date'
})
fig.show()

### b). Top 25 Modules used Pie Chart


In [221]:
modules_count = merged_df_exploded.groupby('modules')['modules'].count().rename('Count').reset_index()
top25_modules = modules_count.nlargest(25,'Count')

fig = px.pie(top25_modules, names = 'modules', values = 'Count', title = 'Top 25 Modules 2024')
fig.show()

### c). Top 25 Modules used Bar Chart

In [222]:
# Most load top 25 from previous cell
fig = px.bar(top25_modules, x = 'modules', y = 'Count', title = 'Top 25 Modules in 2024', labels = {
   'modules':'Modules'
})
fig.show()

### d). Top 25 Module Usage by Day

In [223]:

top25_modules_dates = merged_df_exploded[merged_df_exploded['modules'].isin(top25_modules['modules'])]
top25_modules_dates = top25_modules_dates.groupby(['date','modules'])['modules'].count().rename('Count').reset_index()
fig = px.bar(top25_modules_dates, x = 'date', y = 'Count', color = 'modules', title = 'Top 25 Modules 2024',
             labels={'date': 'Date', 'Count': 'Usage Count', 'modules': 'Module'})
fig.show()



# 6.5 Science Viz

### a). Science Pie chart

In [224]:
science_count = merged_df.groupby('SCIENCE_FIELD_SHORT')['SCIENCE_FIELD_SHORT'].count().rename('Count').reset_index()

fig = px.pie(science_count, names = 'SCIENCE_FIELD_SHORT', values = 'Count', title = 'Science Fields 2024', labels ={
   'SCIENCE_FIELD_SHORT':'Science Field'
})
fig.show()

### b). Science Field by Day

In [225]:
science_count_dates = merged_df.groupby(['SCIENCE_FIELD_SHORT','date'])['SCIENCE_FIELD_SHORT'].count().rename('Count').reset_index()
fig = px.bar(science_count_dates, x = 'date', y = 'Count', color = 'SCIENCE_FIELD_SHORT', title = 'Science Field Count Each Day 2024', labels = {
   'date': 'Date',
   'SCIENCE_FIELD_SHORT': 'Science Field'
})
fig.show()

### c). Science Field by Conda ENV

##### Bar Chart

In [226]:
science_conda = merged_df.groupby(['SCIENCE_FIELD_SHORT','conda'])['SCIENCE_FIELD_SHORT'].count().rename('Count').reset_index()
fig = px.bar(science_conda, x = 'SCIENCE_FIELD_SHORT', y = 'Count', color = 'conda', color_discrete_map ={
   'Yes':'blue',
   'No':'red'
}, title = 'Science Field with Conda Env Count 2024', labels = {
   'SCIENCE_FIELD_SHORT':'Science Field',
   'conda':'Conda'
})
fig.show()

##### Sunburst

In [227]:
fig = px.sunburst(science_conda, path = ['SCIENCE_FIELD_SHORT','conda'], values = 'Count', 
title = 'Science Field with Conda Env Count 2024',labels = {
   'SCIENCE_FIELD_SHORT':'Science Field',
   'conda':'Conda'
})
fig.show()

### d). Science by Node size

#### Bar Chart

In [228]:
science_count_node = merged_df.groupby(['SCIENCE_FIELD_SHORT','node_size'])['SCIENCE_FIELD_SHORT'].count().rename('Count').reset_index()
fig = px.bar(science_count_node, x = 'SCIENCE_FIELD_SHORT', y = 'Count' , color = 'node_size' , title = 'Node Size for Science Domain', labels = {
   'SCIENCE_FIELD_SHORT':'Science Field',
   'node_size':'Node Size'
})
fig.show()

#### Sunburst

In [229]:
fig = px.sunburst(science_count_node, path = ['SCIENCE_FIELD_SHORT','node_size'], values = 'Count', title = 'Node Size for Science Domain', labels = {
   'SCIENCE_FIELD_DOMAIN':'Science Field',
   'node_size':'Node Size'
})
fig.show()

### e). Science by Category

#### Bar chart

In [230]:
science_count_category = merged_df_exploded.groupby(['SCIENCE_FIELD_SHORT','category'])['SCIENCE_FIELD_SHORT'].count().rename('Count').reset_index()
fig = px.bar(science_count_category, x = 'SCIENCE_FIELD_SHORT', y = 'Count', color = 'category', title = 'Categories for Science Domain', labels = {
   'SCIENCE_FIELD_SHORT':'Science Field',
   'category':'Categories'
})
fig.show()


#### Sunburst

In [231]:
fig = px.sunburst(science_count_category, path = ['SCIENCE_FIELD_SHORT','category'], values = 'Count', title = 'Categories for Science Domain')
fig.show()

### e). Science by Modules (Top 10 non-Stdlibs)

#### Bar Chart

In [232]:
science_count_modules = merged_df_exploded[merged_df_exploded['category'] != 'stdlibs']
science_count_modules = science_count_modules.groupby(['SCIENCE_FIELD_SHORT','modules'])['SCIENCE_FIELD_SHORT'].count().rename('Count').reset_index().sort_values(['SCIENCE_FIELD_SHORT', 'Count'], ascending = [True, False]).groupby('SCIENCE_FIELD_SHORT').head(10)


fig = px.bar(science_count_modules, x = 'SCIENCE_FIELD_SHORT', y = 'Count', color = 'modules', title = 'Top 10 Modules for each Science(non-Stdlibs)')
fig.show()
science_count_modules

,SCIENCE_FIELD_SHORT,modules,Count
425,Biological Sciences,ruamel,86183
93,Biological Sciences,cloudpickle,63454
522,Biological Sciences,typing_extensions,50575
434,Biological Sciences,scipy,48391
23,Biological Sciences,_csparsetools,48373
...,...,...,...
7195,Training,sorter,8960
7247,Training,typing_extensions,4729
6950,Training,dill,4063
6891,Training,charset_normalizer,3963


#### Sunburst

In [233]:
fig = px.sunburst(science_count_modules, path = ['SCIENCE_FIELD_SHORT', 'modules'], values = 'Count', title = 'Top 10 Modules for each Science(non-Stdlibs)')
fig.show()